In [2]:
#!pip install natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 165 kB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 41 kB 104 kB/s 
     |████████████████████████████████| 8.2 MB 58.3 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26118 sha256=550a2b9e1f9b66507a01a854ca6798bcb748455f125f92381f132321c1a955ef
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=65845dba7814596c8efe6d2b1abfb21c0fec61296008b72ce0905be7de5cc601
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built intervaltree docopt
  Attempting uninstall: intervaltree
    Fou

In [3]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_excel('/content/Мандарин_reviews.xlsx')

In [7]:
data.head(5)

,Unnamed: 0,author,status,rating,publication_date,review
0,0,Наталья Терентьева,Знаток города 9 уровня,5,2022-08-27,Достойное атмосферное место на любой кошелёк. ...
1,1,Svetlana,Знаток города 11 уровня,4,2022-07-11,Хорошее место для данной локации. Много различ...
2,2,Андрей Белоусов,Знаток города 9 уровня,5,2022-07-02,Рекомендую для прогулки особенно в вечернее вр...
3,3,Ольга Березина,Знаток города 6 уровня,5,2022-09-01,Шикарное место для отдыха и тусовки.Рестораны ...
4,4,Ольга,Знаток города 8 уровня,5,2022-08-09,"Шикарное атмосферное место! Если в Адлер, то т..."


In [17]:
doc = Doc(data.review[4])
doc.segment(segmenter) 
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [18]:
display(doc.tokens[:10])

[DocToken(stop=8, text='Шикарное', id='1_1', head_id='1_3', rel='amod', pos='ADJ', feats=<Nom,Pos,Neut,Sing>),
 DocToken(start=9, stop=20, text='атмосферное', id='1_2', head_id='1_3', rel='amod', pos='ADJ', feats=<Nom,Pos,Neut,Sing>),
 DocToken(start=21, stop=26, text='место', id='1_3', head_id='1_0', rel='root', pos='NOUN', feats=<Inan,Nom,Neut,Sing>),
 DocToken(start=26, stop=27, text='!', id='1_4', head_id='1_3', rel='punct', pos='PUNCT'),
 DocToken(start=28, stop=32, text='Если', id='2_1', head_id='2_3', rel='mark', pos='SCONJ'),
 DocToken(start=33, stop=34, text='в', id='2_2', head_id='2_3', rel='case', pos='ADP'),
 DocToken(start=35, stop=40, text='Адлер', id='2_3', head_id='2_11', rel='nmod', pos='PROPN', feats=<Inan,Acc,Masc,Sing>),
 DocToken(start=40, stop=41, text=',', id='2_4', head_id='2_3', rel='punct', pos='PUNCT'),
 DocToken(start=42, stop=44, text='то', id='2_5', head_id='2_8', rel='cc', pos='SCONJ'),
 DocToken(start=45, stop=51, text='только', id='2_6', head_id='2_8', 

In [19]:
doc.spans

[DocSpan(start=35, stop=40, type='LOC', text='Адлер', tokens=[...]),
 DocSpan(start=54, stop=63, type='PER', text='Мандарину', tokens=[...]),
 DocSpan(start=193, stop=204, type='ORG', text='Перекресток', tokens=[...])]

In [10]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)

names_dict = {_.normal: _.fact.as_dict for _ in doc.spans if _.fact}
names_dict

{}

In [64]:
names_dict_new = {}
addr_dict = {}
facts = []
names = []
addr = []
for doc in data.review:

    matches = dates_extractor(doc)
    facts_n = [i.fact.as_json for i in matches]
    facts = [*facts, *facts_n]

    address = addr_extractor(doc)
    address_n = [i.fact.as_json for i in address]
    addr = [*addr, *address_n]

    names_n = names_extractor(doc)
    names_list = [i.fact.as_json for i in names_n]
    names = [*names, *names_list]

    #doc = Doc(doc)
    #doc.segment(segmenter) 
    #doc.tag_morph(morph_tagger)
    #doc.parse_syntax(syntax_parser)
    #doc.tag_ner(ner_tagger)

    #for span in doc.spans:
    #    span.normalize(morph_vocab)
    #    if span.type == PER:
    #        span.extract_fact(names_extractor)
    #    if span.type == LOC:
    #        span.extract_fact(addr_extractor)

    

    #names_dict_new = {_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}
    #addr_dict_new = {_.normal: _.fact.as_dict for _ in doc.spans if _.type == LOC}
    #names_dict.update(names_dict_new)
    #addr_dict.update(addr_dict_new)

In [58]:
facts_year = []
for i in range(len(facts)):
    try:
        facts_year.append(facts[i]['year'])
    except:
        next

In [60]:
facts_year = list(set(facts_year))
facts_year

[2017, 2020, 2021]

In [ ]:
names_all = []
for i in range(len(names)):
    try:
        names_all.append(names[i]['last'])
        names_all.append(names[i]['first'])
    except:
        next
names_all = list(set(names_all))
names_all

In [66]:
addr_all = []
for i in range(len(addr)):
    try:
        addr_all.append(addr[i]['value'])
        addr_all.append(addr[i]['type'])
    except:
        next
addr_all = list(set(addr_all))
addr_all

['проспект',
 'Были',
 'посёлок',
 'России',
 'Центральной',
 'Мандариновая',
 'проезд',
 'Краснодар',
 'набережная',
 'Красивая',
 'сочи',
 'Всё',
 'Классная',
 'Адлера',
 'Комфортная',
 'Обычная',
 'Европы',
 'Хорошая территория',
 'Москва',
 'находка',
 'улица',
 'аз',
 'С',
 'Огромная',
 'страна',
 'Сочи',
 'Москвы',
 'город',
 'село',
 'площадь']